In [22]:
import subprocess
import json
from constants import *
from dotenv import load_dotenv
from web3 import Web3
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
import os

from web3.middleware import geth_poa_middleware

In [23]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

w3.middleware_onion.inject(geth_poa_middleware, layer=0)


In [24]:
load_dotenv()
mnemonic = os.getenv('MNEMONIC' , 'dawn early execute magic lounge share leopard window method venue scan grace trust cart theme')

In [54]:
def derive_wallets(mnemonic, coin, numderive):
    command = f'php ./derive -g --mnemonic="{mnemonic}" --numderive={numderive} --coin={coin} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    #keys = json.loads(output)
    print(output)

In [26]:
print(mnemonic)

dawn early execute magic lounge share leopard window method venue scan grace trust cart theme


In [50]:
coins = ["btc-test", 'eth', 'btc']

In [53]:
derive_wallets(mnemonic, 'BTC', 3)

b"\nUncaught Exception. code: 0, message: Call to undefined function Mdanter\\Ecc\\Curves\\gmp_init()\n/Users/gmmundh/Desktop/Block Chain Tools/wallet/hd-wallet-derive/vendor/mdanter/ecc/src/Curves/SecgCurve.php : 122\n\nStack Trace:\n#0 /Users/gmmundh/Desktop/Block Chain Tools/wallet/hd-wallet-derive/vendor/mdanter/ecc/src/Curves/SecgCurve.php(137): Mdanter\\Ecc\\Curves\\SecgCurve->curve256k1()\n#1 /Users/gmmundh/Desktop/Block Chain Tools/wallet/hd-wallet-derive/vendor/bitwasp/bitcoin/src/Bitcoin.php(45): Mdanter\\Ecc\\Curves\\SecgCurve->generator256k1()\n#2 /Users/gmmundh/Desktop/Block Chain Tools/wallet/hd-wallet-derive/vendor/bitwasp/bitcoin/src/Bitcoin.php(58): BitWasp\\Bitcoin\\Bitcoin::getGenerator()\n#3 /Users/gmmundh/Desktop/Block Chain Tools/wallet/hd-wallet-derive/vendor/bitwasp/bitcoin/src/Key/Factory/HierarchicalKeyFactory.php(47): BitWasp\\Bitcoin\\Bitcoin::getEcAdapter()\n#4 /Users/gmmundh/Desktop/Block Chain Tools/wallet/hd-wallet-derive/src/WalletDerive.php(58): BitWas

In [39]:
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [40]:
priv_key_to_account("BTCTEST", "cPQvEkVZUVEXwmouobEDHFYD1C7Hpyc7Q39WMiReS7HYy2u9e6HV")

In [41]:
def create_tx(coin, account, to, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas({"from":eth_acc.address, "to": to, "value": amount})
        
        return{
            "from": eth_acc.address,
            "to" : to,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(eth_acc.address)
        }
    
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [55]:
create_tx("BTCTEST", "mxWw3xF5hHtbkKwv8DZoqmwb8ooKWLqtJz", "mmZmHosiund4SZvAFGcLMgrh9pamzCumeS", 0.0001)

In [56]:
def send_tx(coin, account, to, amount):
    raw_tx = create_tx(coin, account, to, amount)
    if coin == ETH:
        tx_sign = eth.acc.sign_transaction(raw_tx)
        result = w3.eth.sendRawTransaction(tx_sign.rawTransaction)
        print(result.hex())
        return result.hex()
    elif coin == BTCTEST:
        tx_sign = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(tx_sign)